## Library Import

In [15]:
import duckdb
import polars as pl
import dotenv
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()

/Users/yiukitcheung/Documents/Projects/condvest/data_pipeline/jupyter_notebook


## Load Gold Data

In [18]:
# Test connection to Silver Data db
try:
    parquet_path = settings['process']['gold_data_path']
    print(f"Attempting to connect to: {parquet_path}")
    # Test Connection with a simple query
    pl_df = pl.read_parquet(parquet_path)
    pl_df = pl_df.sort(["date"], descending=True)
    print("Connection Succesfful!")
    print(f"Test query result: {pl_df}")
    
except Exception as e:
    print(f"Connection Failed {str(e)}")

Attempting to connect to: process/storage/gold/gold_data.parquet
Connection Failed No such file or directory (os error 2): process/storage/gold/gold_data.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'



In [35]:
# Get the size of the gold data
# Get file size in bytes
file_size_bytes = os.path.getsize(parquet_path)

# Convert to more readable format
def convert_size(size_bytes):
    """Convert bytes to human readable format"""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.2f} GB"

print(f"Gold data file size: {convert_size(file_size_bytes)}")
print(f"Number of rows: {len(pl_df)}")
print(f"Number of columns: {len(pl_df.columns)}")


Gold data file size: 2.90 MB
Number of rows: 1257
Number of columns: 5


## Set to Redis

In [36]:
# Get the last row of the gold data
long_accelerating = pl_df.tail(1).select(["long_accelerating"]).to_dicts()[0]["long_accelerating"]
print(long_accelerating)
print(len(long_accelerating))

# Cache long-accelerating to Redis

['BLK', 'CHT', 'VATE', 'VKTX', 'CHY', 'CI', 'CHWY', 'FMS', 'NVMI', 'ABT', 'DSGX', 'KEN', 'ZG', 'AMP', 'ALRM', 'AMD', 'META', 'VIRT', 'ZBRA', 'AAON', 'HASI', 'SCL', 'PRGO', 'DKL', 'TAL', 'SWTX', 'DQ', 'APDN', 'BILI', 'CLW', 'VEEV', 'DRS', 'MBOT', 'NVAX', 'JD', 'FITBI', 'SCPH', 'CHPT', 'ANGI', 'CLLS', 'AAPL', 'MHLD', 'HMC', 'FR', 'BL', 'VERI', 'SCNX', 'IZEA', 'JOUT', 'AMRC', 'SFIX', 'XYZ', 'POWI', 'AAMI', 'FI', 'YUMC', 'ALTS', 'APD', 'FFIV', 'DT', 'TAK', 'DGX', 'YEXT', 'SBAC', 'VHC', 'VIVK', 'BLD', 'FMC', 'AMAT', 'SCNI', 'FF', 'FOLD', 'VIR', 'BIPC', 'MDB', 'VIVS', 'MBUU', 'TEL', 'DOMO', 'AMPH', 'MGNX', 'MCRI', 'ABUS', 'PODD', 'SYRE', 'NTES', 'SCCO', 'DOCU', 'TER', 'TFX', 'FLWS', 'BMY', 'HLF', 'MCK', 'SGRY', 'BLFS', 'TEAM', 'ZWS', 'SDGR', 'PRAA', 'VRNS', 'POWW', 'AMT', 'DKNG', 'K', 'MEIP', 'AMCR', 'TBPH', 'DRD', 'AMH', 'BLDP', 'ZTO', 'PRI', 'BKYI', 'PLD', 'ZM', 'AAP', 'JNPR', 'ADBE', 'VC', 'XYLO', 'ACM', 'NTLA', 'CLWT', 'CLS', 'BMRN', 'ACN', 'JOYY', 'ANSS', 'DLHC', 'DNLI', 'VCYT', 'CLDX',